In [1]:
import plotly.express as px
import pandas as pd
import math

In [2]:
m_b = 5 ** 3 # = 125
m_s = 77 ** 2 # = 5929
min_delta_b = 10000
min_delta_s = 500
max_delta_b = 100000
max_delta_s = 625

In [3]:
perfect = []
best_imperfect = []
other_imperfect = []
delta_b_too_low = []
suboptimal_delta_b = []
prev_delta_b = None
best_imperfect_price = None
best_imperfect_interval = []
for delta_s in range(min_delta_s, max_delta_s + 1):
    delta_b = math.floor(delta_s * m_s / m_b)
    price = delta_s / delta_b
    if (delta_b > max_delta_b):
        delta_b = max_delta_b
    if (delta_b < min_delta_b):
        delta_b_too_low.append((delta_b, delta_s, price))
        continue
    if (delta_b == prev_delta_b):
        suboptimal_delta_b.append((delta_b, delta_s, price))
        continue
    prev_delta_b = delta_b
    if delta_b * m_b == delta_s * m_s:
      perfect.append((delta_b, delta_s, price))
      best_imperfect += best_imperfect_interval
      best_imperfect_price = None
      best_imperfect_interval = []
    elif (best_imperfect_price is None):
      best_imperfect_price = price
      best_imperfect_interval.append((delta_b, delta_s, price))
    elif (price == best_imperfect_price):
      best_imperfect_interval.append((delta_b, delta_s, price))
    elif (price < best_imperfect_price):
      best_imperfect_price = price
      other_imperfect += best_imperfect_interval
      best_imperfect_interval = [(delta_b, delta_s, price)]
    else:
      other_imperfect.append((delta_b, delta_s, price))
    if (delta_b == max_delta_b):
        break
    
best_imperfect += best_imperfect_interval

In [7]:
# Convert your data lists to Pandas DataFrames for easier handling
df1 = pd.DataFrame(perfect, columns=['delta_b', 'delta_s', 'price'])
df2 = pd.DataFrame(best_imperfect, columns=['delta_b', 'delta_s', 'price'])
df3 = pd.DataFrame(other_imperfect, columns=['delta_b', 'delta_s', 'price'])
df4 = pd.DataFrame(delta_b_too_low, columns=['delta_b', 'delta_s', 'price'])
df5 = pd.DataFrame(suboptimal_delta_b, columns=['delta_b', 'delta_s', 'price'])

# Combine the dataframes with an identifier column
df1['Series'] = 'Without rounding errors'
df2['Series'] = 'With rounding errors (best)'
df3['Series'] = 'With rounding errors (other)'
df4['Series'] = 'Delta_b too low'
df5['Series'] = 'Delta_b suboptimal'
combined_df = pd.concat([df1, df2, df3, df4, df5])

# Create an interactive scatter plot
fig = px.scatter(combined_df, x='delta_s', y='price', color='Series',
                 hover_data=['delta_b', 'delta_s', 'price'])

# Add plot titles and labels
fig.update_layout(title='effective price by delta selling',
                  xaxis_title='Delta_s',
                  yaxis_title='Price',
                  height=1600)

# fig.update_yaxes(range=[combined_df['price'].min(), combined_df['price'].max()])  # Adjust the range as needed

# Sort the DataFrame by 'ratio' in descending order
# sorted_df = combined_df.sort_values(by='price', ascending=False)

# Adding a line that connects the dots in decreasing order of their y-value
# fig.add_trace(px.line(sorted_df, x='delta_b', y='price').data[0])

# Show the plot
fig.show()


## We note

### certain periodicities:

those staggered "downward-slopes":
- repeat with multiples of 7 delta_selling, which is the smallest prime factor of m_s
- they repeat 6 times
- the last "lowest point" and the first on of each of those "series" differ by 2

those staggered "upwards-slopes":
- are 3 per perfect-optimum-interval
- the optima are at the beginning of one
- their points repeat with 7 = m_s delta_selling

### and:
- it appears the bounds only make the interval smaller, but don't remove anything from within
  - → this means: if we don't find a perfect solution, we can assume we are within one of those perfect-solution-intervals
- perfect-solution-intervals are all the same size (125 = m_b delta_selling)
- the best imperfect solution per perfect-solution-interval is always offset by 44 delta_selling from the beginning of that interval
  - 44 = 5 * 7 - 1, where 7 is the smallest prime-factor of m_s and 5 the smallest prime-factor of m_b
- the second best is offset by 88 = 44 * 2
- 7 = 44 * 3 - 125 (44 * 3 = 132)
- -> it appears we can get increasing effective prices within such an interval by
  - iterating over those upwards-slopes, which are staggered by 44 delta_selling
  - see if their first value is within our delta-intervals -> if it is, we got the best solution
  - if it is not, increase the point by 7 (for next time it is visited), and continue with the next slope
- we can further optimize it by (note: didn't work so well):
  - let "start" and "end" be the delta_sellings for the perfect solutions defining the interval in question
  - let "current" = "start"
  - repeat the following until the first valid delta_selling is found and return it as best solution:
    - if current is too small (regarding delta_selling or delta_buying or both): 
      - current += 44
    - if current is too big (regarding delta_selling or delta_buying or both): 
      - current -= 125

In [5]:
44 * 3

132